# Оглавление

* [Построение предсказательных моделей](#1)

* [Подготовка данных](#2)

	* [Кодирование ](#2-1)

	* [Нормирование данных](#2-2)

* [Отбор признаков ](#3)

	* [Классификатор RandomForestClassifier](#3-1)

	* [Одномерный отбор признаков](#3-2)

	* [Рекурсивное исключение признаков](#3-3)

* [DecisionTreeClassifier](#4)

* [Метрики качества модели](#5)

	* [Метрики бинарной классификации](#5-1)

	* [Базовые метрики](#5-2)

	* [Кривые точности и полноты ](#5-3)

	* [ROC и AUC](#5-4)

	* [Метрики многоклассовой классификации](#5-5)

* [Оптимизация моделей](#6)

	* [Подготовим данные](#6-1)

	* [Для RandomForest](#6-2)

	* [Для SVM](#6-3)

	* [Логистическая регрессия](#6-4)

	* [MLPClassifier](#6-5)

* [Больше алгоритмов](#7)

	* [Линейные алгоритмы](#7-1)

	* [Нелинейные алгоритмы](#7-2)

	* [Ансамблевые алгоритмы](#7-3)

* [Задача линейной регрессии](#8)

	* [Линейные алгоритмы](#8-1)

	* [Нелинейные алгоритмы](#8-2)

	* [Ансамблевые алгоритмы](#8-3)


# Построение предсказательных моделей<a class="anchor" id="1"></a>

Изначально, нам надо подготовить данные. Ниже год подготовки датасета.

# Подготовка данных<a class="anchor" id="2"></a>

Ниже приведен код с подготовкой данных.

## Загрузка данных

In [ ]:
import sqlalchemy
import pandas as pd
pd.options.display.float_format = '{:,.2f}'.format
import numpy as np

engine = sqlalchemy.create_engine(
                "mysql+pymysql://root:%s@159.69.219.206:3307/rzd" %(sql_pass), encoding='utf8', convert_unicode=True
            )

with engine.connect() as session:
    df=pd.read_sql('SELECT * FROM ku_asrb', con=session)

cChange=['Получили тяжкие телесные повреждения: сотрудников ОАО "РЖД"',
       'Получили тяжкие телесные повреждения: пассажиров',
       'Получили тяжкие телесные повреждения: сторонние',
       'Получили тяжкие телесные повреждения: прочие']
for i in cChange:
    df[i].replace('.','0', inplace=True)

df.loc[:,['Получили тяжкие телесные повреждения: сотрудников ОАО "РЖД"',
       'Получили тяжкие телесные повреждения: пассажиров',
       'Получили тяжкие телесные повреждения: сторонние',
       'Получили тяжкие телесные повреждения: прочие']].fillna(0, inplace=True)
for i in cChange:
    df[i]=df[i].astype(float)
df['Ранено тяжело']=df[cChange].sum(axis=1)

#создадим новой столбец как цель для классификации

#Создадим новый столбец, который и будем предсказывать
df.loc[df['Итоговый суммарный ущерб (тыс.руб.)']=='.', 'Итоговый суммарный ущерб (тыс.руб.)']='0'
df['Итоговый суммарный ущерб (тыс.руб.)']=df['Итоговый суммарный ущерб (тыс.руб.)'].astype(float)
df['Ущерб']=df['Итоговый суммарный ущерб (тыс.руб.)'].apply(lambda x: 1 if x>20 else 0)

##Номер пути
df['Номер пути'].replace('.','0', inplace=True)
df.loc[df['Номер пути'].isin(['1','2','3','4']), 'Номер пути']=1
df.loc[~df['Номер пути'].isin(['1','2','3','4','0']), 'Номер пути']=2
df['Номер пути']=df['Номер пути'].astype(int)

Y_regr=df['Итоговый суммарный ущерб (тыс.руб.)'].values

In [ ]:
df.drop(columns=['index', 'Уникальный идентификатор события (внутри дороги)',
                'Статус события', 'Дата создания события в системе AC PБ',
                'Функциональный филиал', 'Региональный центр', 
                 'Подразделение', 'ДЗО (организация)', 'ДЗО (предприятие)', 
                 'Сервисная организация', 'Сервисная организация (предприятие)',
                'Классификация нарушения 1 уровень (№344)',
                'Классификация нарушения 2 уровень (№344)',
               'Классификация нарушения 3 уровень (№344)',
               'Причина нарушения 1 уровень', 'Причина нарушения 2 уровень',
               'Причина нарушения 3 уровень', 'Причина нарушения 4 уровень',
                'Погибло сотрудников ОАО "РЖД"', 'Погибло пассажиров',
               'Погибло сторонних', 'Погибло прочих', 'Получили тяжкие телесные повреждения: сотрудников ОАО "РЖД"',
               'Получили тяжкие телесные повреждения: пассажиров',
               'Получили тяжкие телесные повреждения: сторонние',
               'Получили тяжкие телесные повреждения: прочие','Размер возмещенного ущерба (тыс.руб.)',
               'Станция/перегон id', 'Переезд',
               'Станция/перегон id', 'Путь общего/необщего пользования',
               'Пикет', 'Станция/перегон', 'Дорога НБД'], inplace=True)

print('Осталось столбцов', len(df.columns))

## Приведение типов

In [ ]:
def ddate(s):
    try:
        res=pd.to_datetime(s, format='%d%b%Y:%H:%M:%S')
    except:
        res=np.NaN
    return res
df['Дата события']=df['Дата события'].apply(lambda x: ddate(x))
df['Дата события'].dropna(inplace=True)

#создадим еще два признака - номер месяца и день недели
df['Месяц события']=df['Дата события'].apply(lambda x: x.month)
df['День недели']=df['Дата события'].apply(lambda x: x.isoweekday())
df['Время суток']=df['Дата события'].apply(lambda x: 1 if 8<x.hour<20 else 0)

#время
def time_to_sec(s):
    try:
        s3=s.split(':')
    except:
        return 0
    
    if len(s3)<3:
        return 0
    else:
        return int(s3[0])*60*60+int(s3[1])*60+int(s3[2])

df['Время полного перерыва движения']=df['Время полного перерыва движения'].apply(lambda x: time_to_sec(x))
df['Время полного перерыва движения']=df['Время полного перерыва движения'].apply(lambda x: 1 if x>0 else 0)
df['Количество задержанных поездов']=df['Количество задержанных поездов'].apply(lambda x: time_to_sec(x))

#заполним пропуски
df['Тип виновного предприятия'].fillna('Неизвестно', inplace=True)
df['Сторонняя организация'].fillna('Неизвестно', inplace=True)

#переведем в целочисленный тип
df['Погибло всего'].replace('.',0, inplace=True)
df['Погибло всего']=df['Погибло всего'].astype(int)

df['Ранено легко'].replace('.',0, inplace=True)
df['Ранено легко']=df['Ранено легко'].astype(int)

df['Километр'].replace('.',0, inplace=True)
df['Километр']=df['Километр'].astype(int)

df['Общее время задержки'].replace('.',0, inplace=True)
df['Общее время задержки']=df['Общее время задержки'].astype(int)

df['Регион'].fillna('Неизвестен', inplace=True)

#проверим на пропуски
print(df.isna().sum())

#удалим строки с пропусками
df.dropna(axis='index', inplace=True)

df.describe()

## Удалим выбросы

In [ ]:
df.drop(np.where(df['Итоговый суммарный ущерб (тыс.руб.)']>30000)[0], inplace=True)
df=df[df['Общее время задержки']<18]
df=df[df['Количество задержанных поездов']<73000]

## Кодирование <a class="anchor" id="2-1"></a>

Выполним кодирование категориальных переменных. Закодируем их через LabelEncoder. Такой вариант подходит для классификации, но не подходит для задачи регрессии.

In [ ]:
#Количественные столбцы
object_columns=df.select_dtypes(include = ['object']).columns

int_columns=list(set(df.columns)-set(object_columns))

print('Применим LabelEncoder к столбцам:')
[print('-',i) for i in object_columns]

from sklearn.preprocessing import LabelEncoder
from collections import defaultdict

d = defaultdict(LabelEncoder)

#Перекодируем все столбцы
data = df[object_columns].apply(lambda x: d[x.name].fit_transform(x))

for i in int_columns:
    data[i]=df[i]
    
#из даты события можно получить еще ряд признаков
data.drop(columns=['Дата события'], inplace=True)
data.sample(3)

In [ ]:
data.info()

## Нормирование данных<a class="anchor" id="2-2"></a>

Воспользуемся самым простым подходом к нормированию данных Мин-Макс скалером. Помним, что в переменной data нам всегда доступны оригинальные данные. 

Также для применения другого скалера надо перезапустить этот код.

In [ ]:
from sklearn import preprocessing

Y=data['Ущерб'].values
df_X=data.drop(columns=['Ущерб', 'Итоговый суммарный ущерб (тыс.руб.)'])

X = preprocessing.MinMaxScaler().fit_transform(df_X)
X=pd.DataFrame(X, columns=df_X.columns)

del df #удалим ненужные таблицы из памяти

from sklearn.model_selection import train_test_split
#разделим набор на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

# Отбор признаков <a class="anchor" id="3"></a>

Интуитивно и ошибочно мы считаем, что чем больше признаков подадим на вход модели обучения, тем лучше. Увы, часто это не так. Данные могут неправильно взаимодействовать между собой, дублировать, не содержать полезных сигналов, усложнять обучение модели и резко увеличивать ее объем. Прибегнем к нескольким способам оценить полезность признаков нашей модели. 

**Обратите внимание:** так как для отбора используются алгоритмы классификации, значения целевой переменной приводятся к целым числам. Если дробная часть значений целевой переменной играет существенную роль, то необходимо или умножить все значения на 100, или преобразовать в категорийные переменные.

## Классификатор RandomForestClassifier<a class="anchor" id="3-1"></a>

В этом классе реализована метаоценка, которая подходит к ряду рандомизированных деревьев решений (также называемых дополнительными деревьями) в различных подвыборках набора данных и использует усреднение для повышения точности прогнозирования и контроля соответствия.

Результаты подобных алгоритмов сложно интерпретировать, но они позволяют сделать оценку значимости признаков.

```python
import matplotlib.pyplot as plt
from sklearn.ensemble import RandomForestClassifier

#feature extraction
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X, Y.astype('int'))

importances = model.feature_importances_
indices = np.argsort(importances)[::-1]

ar_f=[]
for f, idx in enumerate(indices):
    ar_f.append([round(importances[idx],4), X.columns[idx]])
print("Значимость признака:")
ar_f.sort(reverse=True)
ar_f
```

Удобнее отобразить результаты в виде графика.

```python
d_first = len(X.columns)
plt.figure(figsize=(8, 5))
plt.title("Значимость признака")
plt.bar(range(d_first), importances[indices[:d_first]], align='center')
plt.xticks(range(d_first), np.array(X.columns)[indices[:d_first]], rotation=90)
plt.xlim([-1, d_first]);
```

Посмотрим на метрику точности R2, но надо понимать, что для несбалансированных классов, как у нас, она плохо подходит.

```python
from sklearn.metrics import r2_score
r2_score(model.predict(X), Y) #не совсем точная характеристика!!! 
```

#### Несколько усовершенствованный подход

Более сложный вариант выбора значимых показателей с использованием RandomForestClassifier.

```python
from sklearn.feature_selection import SelectFromModel
from sklearn.ensemble import RandomForestClassifier
select = SelectFromModel(
    RandomForestClassifier(n_estimators=100, random_state=42),
    threshold="median")

select.fit(X, Y)
X_train_l1 = select.transform(X)
print("форма обуч набора X: {}".format(X.shape))
print("форма обуч набора X c l1: {}".format(X_train_l1.shape))

mask = select.get_support()
#визуализируем булевы значения -- черный – True, белый – False
plt.matshow(mask.reshape(1, -1), cmap='gray_r')
plt.xlabel("Индекс примера")
```

Выведем результаты в виде списка.

```python
for i in zip(X.columns, mask):
    print(i[1],'<-', i[0])
```

## Одномерный отбор признаков<a class="anchor" id="3-2"></a>

Признаки, имеющие наиболее выраженную взаимосвязь с целевой переменной, могут быть отобраны с помощью статистических критериев. Библиотека scikit-learn содержит класс SelectKBest, реализующий одномерный отбор признаков (univariate feature selection). Этот класс можно применять совместно с различными статистическими критериями для отбора заданного количества признаков. В нашем случае мы используем критерий chi2.

```python
from sklearn.feature_selection import SelectKBest
from sklearn.feature_selection import chi2

#создаем и тренируем модель
test = SelectKBest(score_func=chi2, k='all')
fit = test.fit(X, Y.astype('int'))

ar2=[]
for i in range(0, len(X.columns)):
    ar2.append([X.columns[i], round(fit.scores_[i],4)])
ar2.sort(key=lambda i: i[1], reverse=True)
df_om=pd.DataFrame(ar2, columns=['Признак', "Значимость"])
print('Значимость признаков в порядке убывания:')
df_om
```

## Рекурсивное исключение признаков<a class="anchor" id="3-3"></a>

Метод рекурсивного исключения признаков (recursive feature elimination, RFE) реализует следующий алгоритм: модель обучается на исходном наборе признаков и оценивает их значимость, затем исключается один или несколько наименее значимых признаков, модель обучается на оставшихся признаках и так далее, пока не останется заданное количество лучших признаков. В документации scikit-learn вы можете подробнее прочитать о классе RFE.

В примере ниже метод RFE применяется в сочетании с логистической регрессией для отбора 4-х лучших признаков. Для совместного использования с RFE можно выбирать различные модели, важно лишь, чтобы они были достаточно эффективны и совместимы с RFE.

Количество отбираемых признаков можно изменить.

```python
from sklearn.feature_selection import RFE
from sklearn.linear_model import LogisticRegression

#создаем модель, которую будем обучать
model = LogisticRegression(solver='liblinear', multi_class='auto', random_state=42)

rfe = RFE(model, 4) #указано количество отбираемых признаков

#обучаем модель
fit = rfe.fit(X, Y.astype('int'))

#выводим результаты
ar3=[]
for i in range(0,len(X.columns)):
    ar3.append([X.columns[i], fit.support_[i], fit.ranking_[i]])
ar3.sort(key=lambda i: i[2])
df_rfe=pd.DataFrame(ar3, columns=["Признак", "Важный", "Рэнк"])
df_rfe
```

# DecisionTreeClassifier<a class="anchor" id="4"></a>

Начнем с одной из интересных моделей DecisionTreeClassifier. Эта модель позволит лучше понять в том числе и RandomForest.

```python
from sklearn.tree import DecisionTreeClassifier
tree = DecisionTreeClassifier(max_depth=3, random_state=0)
tree.fit(X_train, y_train)
print("Правильность на обучающем наборе: {:.3f}".format(tree.score(X_train, y_train)))
print("Правильность на тестовом наборе: {:.3f}".format(tree.score(X_test, y_test)))
print('Важность признака')
for i in zip(X.columns, tree.feature_importances_):
    print("{:.3f}".format(i[1]),'<-',i[0])
```

Мы можем отобразить дерево принятия решения со всеми его ветвями.

```python
from sklearn.tree import export_graphviz
import graphviz

export_graphviz(tree, out_file="tree.dot", class_names=["0", "1"],
feature_names=X.columns, impurity=False, filled=True)

with open("tree.dot") as f:
    dot_graph = f.read()
graphviz.Source(dot_graph)
```

Также можно сохранить в pdf файл.

```python
import pydotplus
from sklearn.tree import export_graphviz
from sklearn import tree
from sklearn.tree import export_graphviz
clf = tree.DecisionTreeClassifier(max_depth=4, random_state=0)
clf = clf.fit(X_train, y_train)
dot_data = tree.export_graphviz(clf, out_file=None)
graph = pydotplus.graph_from_dot_data(dot_data)
graph.write_pdf("Tree.pdf")
```

Или отобразить на экране в виде более аккуратного рисунка.

```python
from IPython.display import Image
dot_data = tree.export_graphviz(clf, out_file=None, feature_names=X.columns,
    class_names=['0','1'],
    filled=True, rounded=True,
    special_characters=True)
graph = pydotplus.graph_from_dot_data(dot_data)
Image(graph.create_png())
```

# Метрики качества модели<a class="anchor" id="5"></a>

Мы говорили, что точность нашей модели несколько неудовлетворительна, а R2 не самая адекватная оценка. 

Давайте обучим и протестируем RandomForestClassifier.

```python
from sklearn.ensemble import RandomForestClassifier
#feature extraction
model = RandomForestClassifier(n_estimators=10, random_state=42)
model.fit(X_train, y_train)

from sklearn.metrics import r2_score
print(r2_score(model.predict(X_train), y_train))
print(r2_score(model.predict(X_test), y_test))
```

Посмотрим на результаты прогноза при помощи confusion_matrix.

```python
#матрица количества правильно и ошибочно угаданных классов
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, model.predict(X_test)), columns=[0,1], index=[0,1])
```

Ее вариант в виде таблицы с процентами. 

```python
#так же матрица в процентах и более изящном виде
matrix = confusion_matrix(y_test, model.predict(X_test))
matrix = matrix.astype('float') / matrix.sum(axis=1)[:, np.newaxis]

#Build the plot
import seaborn as sns
plt.figure(figsize=(7,5))
sns.set(font_scale=1.4)
sns.heatmap(matrix, annot=True, annot_kws={'size':10},
            cmap=plt.cm.Greens, linewidths=0.2)

#Add labels to the plot
class_names = ['0', '1']                 # !!!!!! указать названия классов!
tick_marks = np.arange(len(class_names))
tick_marks2 = tick_marks + 0.5
plt.xticks(tick_marks, class_names, rotation=25)
plt.yticks(tick_marks2, class_names, rotation=0)
plt.xlabel('Предсказанные классы')
plt.ylabel('Истинные классы')
plt.title('Confusion Matrix for Random Forest Model')
plt.show()
```

Выбирая метрику, вы всегда должны помнить о конечной цели проекта машинного обучения. На практике мы, как правило, заинтересованы не только в создании точных прогнозов, но и в том, чтобы использовать их в рамках более масштабного процесса принятия решений. Прежде чем выбрать показатель качества машинного обучения, вам стоит подумать о высокоуровневой цели вашего проекта, которую часто называют бизнес-метрикой (business metric). Последствия, обусловленные выбором конкретного алгоритма для того или иного проекта, называются влиянием на бизнес (business impact). Например, высокоуровневой целью является предотвращение дорожно-транспортных происшествий или уменьшение числа случаев госпитализации. Вы должны выбрать такую модель или такие значения
параметров, которые оказывают наибольшее положительное влияние на
бизнес-метрику. Часто эта задача является трудной, поскольку оценка
влияния конкретной модели на бизнес может потребовать ее внедрения
в реальное производство.

## Метрики бинарной классификации<a class="anchor" id="5-1"></a>

Бинарная классификация является, пожалуй, наиболее распространенным и концептуально простым примером практического применения машинного обучения. Однако даже при решении этой простой задачи существует целый ряд нюансов. Прежде чем мы углубимся в альтернативные метрики, давайте рассмотрим ситуации, в которых правильность измерения может ввести в заблуждение. Вспомним, что в случае бинарной классификации мы говорим о положительном (positive) классе и отрицательном (negative) классе, подразумевая под положительным классом интересующий нас класс.

Одна из возможных ошибок заключается в том, что здоровый пациент будет классифицирован как больной (положительный класс), что даст повод для дополнительного тестирования. Дополнительное обследование приведет к некоторым затратам и неудобствам для пациента (и, возможно, к определенному психическому дискомфорту). Пример, неправильно спрогнозированный как положительный, называется ложно положительным (false positive). Другая возможная ошибка состоит в том, что больной пациент будет классифицирован как здоровый (отрицательный класс), не пройдет дополнительные тесты и не получит лечения. Недиагностированный вовремя рак может привести к серьезным проблемам со здоровьем и может даже закончиться смертельным исходом. Пример, неправильно спрогнозированный как отрицательный, называется ложно отрицательным (false negative). В статистике ложно положительный пример также известен как ошибка I 
рода (type I error), а ложно отрицательный пример – как ошибка II рода (type II error). Мы будем придерживаться определений «ложно отрицательный пример» и «ложно положительный пример», поскольку они являются более явными и их легче запомнить. В примере с диагностикой рака очевидно, что мы хотим минимизировать долю ложно отрицательных примеров, тогда как ложно положительные примеры можно считать гораздо менее значительной неприятностью.

## Базовые метрики<a class="anchor" id="5-2"></a>

![Матрица ошибок](https://miro.medium.com/max/356/1*g5zpskPaxO8uSl0OWT4NTQ.png)

### Accuracy

Интуитивно понятной, очевидной и почти неиспользуемой метрикой является accuracy — доля правильных ответов алгоритма:

Эта метрика бесполезна в задачах с неравными классами, и это легко показать на примере.

**Accuracy (все правильные / все) = TP + TN / TP + TN + FP + FN**

Допустим, мы хотим оценить работу спам-фильтра почты. У нас есть 100 не-спам писем, 90 из которых наш классификатор определил верно (True Negative = 90, False Positive = 10), и 10 спам-писем, 5 из которых классификатор также определил верно (True Positive = 5, False Negative = 5).

Тогда: accuracy=(5+90)/(5+90+10+5)=0.864

Однако если мы просто будем предсказывать все письма как не-спам, то получим более высокую accuracy:

accuracy=(0+100)/(0+100+0+10)=0.909

При этом наша модель совершенно не обладает никакой предсказательной силой, так как изначально мы хотели определять письма со спамом. Преодолеть это нам поможет переход с общей для всех классов метрики к отдельным показателям качества классов.

### Precision, recall и F-мера

Для оценки качества работы алгоритма на каждом из классов по отдельности введем метрики precision (точность) и recall (полнота).

**Precision (правильные позитивные / на предсказанные как позитивные) = TP / TP + FP**

**Sensitivity aka Recall (правильные позитивные / на все настоящие позитивные) = TP / TP + FN**

Precision можно интерпретировать как долю объектов, названных классификатором положительными и при этом действительно являющимися положительными, а recall показывает, какую долю объектов положительного класса из всех объектов положительного класса нашел алгоритм.

Именно введение precision не позволяет нам записывать все объекты в один класс, так как в этом случае мы получаем рост уровня False Positive. Recall демонстрирует способность алгоритма обнаруживать данный класс вообще, а precision — способность отличать этот класс от других классов.

Как мы отмечали ранее, ошибки классификации бывают двух видов: False Positive и False Negative. В статистике первый вид ошибок называют ошибкой I-го рода, а второй — ошибкой II-го рода. В нашей задаче, например, по определению оттока абонентов ошибкой первого рода будет принятие лояльного абонента за уходящего, так как наша нулевая гипотеза состоит в том, что никто из абонентов не уходит, а мы эту гипотезу отвергаем. Соответственно, ошибкой второго рода будет являться «пропуск» уходящего абонента и ошибочное принятие нулевой гипотезы.
Precision и recall не зависят, в отличие от accuracy, от соотношения классов и потому применимы в условиях несбалансированных выборок.
Часто в реальной практике стоит задача найти оптимальный (для заказчика) баланс между этими двумя метриками. Классическим примером является задача определения оттока клиентов.

Очевидно, что мы не можем находить всех уходящих в отток клиентов и только их. Но, определив стратегию и ресурс для удержания клиентов, мы можем подобрать нужные пороги по precision и recall. Например, можно сосредоточиться на удержании только высокодоходных клиентов или тех, кто уйдет с большей вероятностью, так как мы ограничены в ресурсах колл-центра.
Обычно при оптимизации гиперпараметров алгоритма (например, в случае перебора по сетке GridSearchCV) используется одна метрика, улучшение которой мы и ожидаем увидеть на тестовой выборке.

Хотя точность и полнота являются очень важными метриками, сами
по себе они не дадут вам полной картины. Одним из способов
подытожить их является F-мера (F-measure), которая представляет собой
гармоническое среднее точности и полноты:

**F1=2 x (precision x recall)/(precision + recall)**

Поскольку F1-мера учитывает точность и полноту, то для бинарной классификации несбалансированных данных она может быть лучшей метрикой, чем правильность.

## Кривые точности и полноты <a class="anchor" id="5-3"></a>

Изменение порога, используемого для классификации решений модели – это способ, позволяющий найти компромисс между точностью и полнотой для данного классификатора. Возможно, вы хотите пропустить менее 10% положительных примеров, таким образом, желаемое значение полноты составит 90%. Решение зависит от конкретного примера и оно должно определяться бизнес- целями. Как только поставлена конкретная цель, скажем, задано конкретное значение полноты или точности для класса, можно установить соответствующий порог. Всегда можно задать то или иное пороговое значение для реализации конкретной цели (например, достижения значения полноты 90%). Трудность состоит в разработке такой модели, которая при этом пороге еще и будет иметь приемлемое значение точности, ведь классифицировав все примеры как положительные, вы получите значение полноты, равное 100%, но при этом ваша модель будет бесполезной.

Требование, выдвигаемое к качеству модели (например, значение полноты должно быть 90%), часто называют рабочей точкой (operating point). Фиксирование рабочей точки часто бывает полезно в контексте бизнеса, чтобы гарантировать определенный уровень качества клиентам или другим группам лиц внутри организации.

Как правило, при разработке новой модели нет четкого представления о том, что будет рабочей точкой. По этой причине, а также для того, чтобы получить более полное представление о решаемой задаче, полезно сразу взглянуть на все возможные пороговые значения или все возможные соотношения точности и полноты для этих пороговых значений. Данную процедуру можно осуществить с помощью инструмента, называемого кривой точности-полноты (precision-recall curve). Функцию для вычисления кривой точности-полноты можно найти в модуле sklearn.metrics. Ей необходимо передать фактические метки классов и спрогнозированные вероятности, вычисленные с помощью decision_function или predict_proba:


```python
#в RandomForestClassifier есть predict_proba, но нет decision_function
from sklearn.metrics import precision_recall_curve
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(
y_test, model.predict_proba(X_test)[:, 1])

plt.plot(precision_rf, recall_rf, label="rf")
close_default_rf = np.argmin(np.abs(thresholds_rf - 0.5))
plt.plot(precision_rf[close_default_rf], recall_rf[close_default_rf], '^', c='k',
markersize=10, label="порог 0.5 rf", fillstyle="none", mew=2)
plt.xlabel("Точность")
plt.ylabel("Полнота")
plt.legend(loc="best")
```

Обучим еще одну модель - SVC и оценим ее результаты.

```python
from sklearn.svm import SVC
svc = SVC().fit(X_train, y_train)

from sklearn.metrics import precision_recall_curve
precision, recall, thresholds = precision_recall_curve(
    y_test, svc.decision_function(X_test))

precision, recall, thresholds = precision_recall_curve(
y_test, svc.decision_function(X_test))
close_zero = np.argmin(np.abs(thresholds))
#находим ближайший к нулю порог
#в RandomForestClassifier есть predict_proba, но нет decision_function
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(
y_test, model.predict_proba(X_test)[:, 1])
plt.plot(precision, recall, label="svc")
plt.plot(precision[close_zero], recall[close_zero], 'o', markersize=10,
label="порог 0 svc", fillstyle="none", c='k', mew=2)
plt.xlabel("Точность")
plt.ylabel("Полнота")
plt.legend(loc="best")
```

Матрица результатов предсказаний.

```python
from sklearn.metrics import confusion_matrix
pd.DataFrame(confusion_matrix(y_test, svc.predict(X_test)), columns=[0,1], index=[0,1])
```

Сравним результаты опираясь на график.

```python
from sklearn.metrics import roc_curve

precision, recall, thresholds = precision_recall_curve(
y_test, svc.decision_function(X_test))
close_zero = np.argmin(np.abs(thresholds))
#находим ближайший к нулю порог
#в RandomForestClassifier есть predict_proba, но нет decision_function
precision_rf, recall_rf, thresholds_rf = precision_recall_curve(
y_test, model.predict_proba(X_test)[:, 1])
plt.plot(precision, recall, label="svc")
plt.plot(precision[close_zero], recall[close_zero], 'o', markersize=10,
label="порог 0 svc", fillstyle="none", c='k', mew=2)


plt.plot(precision_rf, recall_rf, label="rf")
close_default_rf = np.argmin(np.abs(thresholds_rf - 0.5))
plt.plot(precision_rf[close_default_rf], recall_rf[close_default_rf], '^', c='k',
markersize=10, label="порог 0.5 rf", fillstyle="none", mew=2)
plt.xlabel("Точность")
plt.ylabel("Полнота")
plt.legend(loc="best")
```

Мы видим, что модель RF работает лучше. Это также очевидно из метрики f1.

```python
from sklearn.metrics import f1_score

print("f1-мера random forest: {:.3f}".format(
f1_score(y_test, model.predict(X_test))))
print("f1-мера svc: {:.3f}".format(f1_score(y_test, svc.predict(X_test))))
```

## ROC и AUC<a class="anchor" id="5-4"></a>

Еще один инструмент, который обычно используется для анализа поведения классификаторов при различных пороговых значениях – это кривая рабочей характеристики приемника (receiver operating characteristics curve) или кратко ROC-кривая (ROC curve). Как и кривая точности-полноты, ROC-кривая позволяет рассмотреть все пороговые значения для данного классификатора, но вместо точности и полноты она показывает долю ложно положительных примеров (false positive rate, FPR) в сравнении с долей истинно положительных примеров (true positive rate). Вспомним, что доля истинно положительных примеров – это просто еще одно название полноты, тогда как доля ложно положительных примеров – это доля ложно положительных примеров от общего количества отрицательных примеров:

FRP=FT/(FP+TN)

```python
from sklearn.metrics import roc_curve
fpr, tpr, thresholds = roc_curve(y_test, svc.decision_function(X_test))
plt.plot(fpr, tpr, label="ROC-кривая")
plt.xlabel("FPR")
plt.ylabel("TPR (полнота)")
#находим пороговое значение, ближайшее к нулю
close_zero = np.argmin(np.abs(thresholds))
plt.plot(fpr[close_zero], tpr[close_zero], 'o', markersize=10,
label="порог 0", fillstyle="none", c='k', mew=2)
plt.legend(loc=4)
```

Как и в случае с кривой точности-полноты, мы хотим подытожить
информацию ROC-кривой с помощью одного числа, площади под кривой
(обычно ее просто называют AUC, при этом имейте в виду, что речь идет
о ROC-кривой). Мы можем вычислить площадь под ROC-кривой с
помощью функции roc_auc_score:

```python
from sklearn.metrics import roc_auc_score
rf_auc = roc_auc_score(y_test, model.predict_proba(X_test)[:, 1])
svc_auc = roc_auc_score(y_test, svc.decision_function(X_test))
print("AUC для случайного леса: {:.3f}".format(rf_auc))
print("AUC для SVC: {:.3f}".format(svc_auc))
```

## Метрики многоклассовой классификации<a class="anchor" id="5-5"></a>

Метрики для классификации более чем двух классов, практически не отличаются.

```python
#загрузим набор данных
from sklearn.metrics import classification_report
from sklearn.datasets import load_digits
from sklearn.linear_model import LogisticRegression

digits = load_digits()
print(digits.data.shape)

from sklearn.metrics import accuracy_score

X_train, X_test, y_train, y_test = train_test_split(
    digits.data, digits.target, random_state=0)

lr = SVC(max_iter=1000).fit(X_train, y_train)

pred = lr.predict(X_test)

print("Accuracy: {:.3f}".format(accuracy_score(y_test, pred)))
print("Confusion matrix:\n{}".format(confusion_matrix(y_test, pred)))
print(classification_report(y_test, pred))
```

# Оптимизация моделей<a class="anchor" id="6"></a>

Каждая модель имеет большое количество параметров. Не всегда удается вручную подобрать их оптимально. В этом случае нам поможет метод GridSearchCV. Так как это достаточно затратная по времени и ресурсам операция, посмотрим на примере двух моделей.

## Подготовим данные<a class="anchor" id="6-1"></a>

```python
from sklearn import preprocessing

Y=data['Ущерб'].values
df_X=data.drop(columns=['Ущерб', 'Итоговый суммарный ущерб (тыс.руб.)'])

X = preprocessing.MinMaxScaler().fit_transform(df_X)
X=pd.DataFrame(X, columns=df_X.columns)

from sklearn.model_selection import train_test_split
#разделим набор на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)
```

## Для RandomForest<a class="anchor" id="6-2"></a>

**Высоконагруженная операция**

```python
from sklearn.model_selection import GridSearchCV

param_grid = { 
    'n_estimators': [10, 50, 100, 200],
    'max_features': ['auto', 'sqrt', 'log2'],
    'max_depth' : [6,8],
}

CV_rfc = GridSearchCV(estimator=model, param_grid=param_grid, scoring='f1', cv= 5)
CV_rfc.fit(X_train, y_train)

print("Наилучшие значения параметров: {}".format(CV_rfc.best_params_))
print("Наилучшее значение кросс-валидац. правильности: {:.2f}".format(CV_rfc.best_score_))
```

## Для SVM<a class="anchor" id="6-3"></a>

**Высоконагруженная операция**

```python
param_grid = {'C': [0.001, 0.01, 0.1, 1, 10, 100, 1000] }
grid_search = GridSearchCV(SVC(), param_grid, scoring='f1', cv=5)
grid_search.fit(X_train, y_train)

print("Наилучшие значения параметров: {}".format(grid_search.best_params_))
print("Наилучшее значение кросс-валидац. правильности {:.2f}".format(grid_search.best_score_))
```

Обучим лучшие модели.

```python
model = RandomForestClassifier( max_depth=8, max_features='auto', n_estimators=200)
model.fit(X_train, y_train)
svc = SVC(C=10).fit(X_train, y_train)

print("f1-мера random forest: {:.3f}".format(f1_score(y_test, model.predict(X_test))))
print("f1-мера svc: {:.3f}".format(f1_score(y_test, svc.predict(X_test))))
```

## Логистическая регрессия<a class="anchor" id="6-4"></a>

```python
from sklearn.linear_model import LogisticRegression

logistic = LogisticRegression(C=0.1)
res=logistic.fit(X_train, y_train)
logistic.score(X_test, y_test)

print(f1_score(y_test, logistic.predict(X_test)))
```

Преимущество модели - это простая интерпретируемость.

```python
for i in zip(X_train.columns, logistic.coef_[0]):
    print(i[0], '->  {:.3f}'.format(i[1]))
    
print('Intercept: ->  ', logistic.intercept_[0])
```

Оценить вероятности принадлежности к классу.

```python
logistic.predict_proba(X)
```

## MLPClassifier<a class="anchor" id="6-5"></a>

Одна из самых простых в реализации нейронных сетей для классификации из Scikit-Learn, которая называется MLPClassifier. MLPClassifier расшифровывается как Multi-Layer Perceptron classifier, который в самом названии указывает на нейронную сеть. В отличие от других алгоритмов классификации, таких как опорные вектора или наивный байесовский классификатор, MLPClassifier полагается на базовую нейронную сеть для выполнения задачи классификации.

Однако одно сходство с другими алгоритмами классификации Scikit-Learn заключается в том, что реализация MLPClassifier требует не больше усилий, чем реализация Support Vectors, Naive Bayes или любых других классификаторов из Scikit-Learn.

- `hidden_layer_sizes`: этот параметр позволяет нам установить количество слоев и количество узлов, которые мы хотим иметь в классификаторе нейронной сети. Каждый элемент в кортеже представляет количество узлов в i-й позиции, где i - это индекс кортежа. Таким образом, длина кортежа обозначает общее количество скрытых слоев в сети.
- `max_iter`: обозначает количество эпох.
- `activation`: функция активации скрытых слоев.
- `solver`: этот параметр определяет алгоритм оптимизации веса для узлов.
- `random_state`: параметр позволяет установить начальное число для воспроизведения тех же результатов.

```python
from sklearn.neural_network import MLPClassifier
mlp = MLPClassifier(solver='lbfgs', random_state=0, max_iter=1000, 
                    hidden_layer_sizes=[20,10, 20], 
                    activation = 'relu', early_stopping=True).fit(X_train, y_train)

print(classification_report(y_test, mlp.predict(X_test)))
```

# Больше алгоритмов<a class="anchor" id="7"></a>

## Линейные алгоритмы<a class="anchor" id="7-1"></a>
- Логистическая регрессия / Logistic Regression (‘LR’). Слово «регрессия» может сбить с толку. Но не забываем, что «Логистическая регрессия» — это алгоритм классификации
- Линейный дискриминантный анализ / Linear Discriminant Analysis (‘LDA’)

## Нелинейные алгоритмы<a class="anchor" id="7-2"></a>
- Метод k-ближайших соседей (классификация) / K-Neighbors Classifier (‘KNN’)
- Деревья принятия решений / Decision Tree Classifier (‘CART’)
- Наивный классификатор Байеса / Naive Bayes Classifier (‘NB’)
- Линейный метод опорных векторов (классификация) / Linear Support Vector Classification (‘LSVC’)
- Метод опорных векторов (классификация) / C-Support Vector Classification (‘SVC’)

## Ансамблевые алгоритмы<a class="anchor" id="7-3"></a>
- Bagging (классификация) / Bagging Classifier (‘BG’) (Bagging = Bootstrap aggregating)
- Случайный лес (классификация) / Random Forest Classifier (‘RF’)
- Экстра-деревья (классификация) / Extra Trees Classifier (‘ET’)
- AdaBoost (классификация) / AdaBoost Classifier (‘AB’) (AdaBoost = Adaptive Boosting)
- Градиентный boosting (классификация) / Gradient Boosting Classifier (‘GB’)

[Развернутый пример](https://habr.com/ru/post/475552/)

In [ ]:
from sklearn.model_selection import train_test_split
from sklearn.linear_model import LogisticRegression
from sklearn.discriminant_analysis import LinearDiscriminantAnalysis
from sklearn.neighbors import KNeighborsClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.naive_bayes import GaussianNB
from sklearn.svm import LinearSVC
from sklearn.svm import SVC
from sklearn.ensemble import BaggingClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.ensemble import ExtraTreesClassifier
from sklearn.ensemble import AdaBoostClassifier
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import Normalizer
from matplotlib import pyplot
from sklearn.metrics import f1_score

#Настройка параметров оценивания алгоритма
seed = 7
num_folds = 10
n_estimators = 100
scoring = 'accuracy'

models = []
models.append(('LR', LogisticRegression()))
models.append(('LDA', LinearDiscriminantAnalysis()))
models.append(('KNN', KNeighborsClassifier()))
models.append(('CART', DecisionTreeClassifier()))
models.append(('NB', GaussianNB()))
models.append(('LSVC', LinearSVC()))
models.append(('SVC', SVC()))
models.append(('BG', BaggingClassifier(n_estimators=n_estimators)))
models.append(('RF', RandomForestClassifier(n_estimators=n_estimators)))
models.append(('ET', ExtraTreesClassifier(n_estimators=n_estimators)))
models.append(('AB', AdaBoostClassifier(n_estimators=n_estimators, algorithm='SAMME')))
models.append(('GB', GradientBoostingClassifier(n_estimators=n_estimators)))

#Оценивание эффективности выполнения каждого алгоритма
scores = []
names = []
results = []
predictions = []
msg_row = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    names.append(name)
    results.append(cv_results)
    m_fit = model.fit(X_train, y_train)
    m_predict = model.predict(X_test)
    predictions.append(m_predict)
    m_score = model.score(X_test, y_test)
    scores.append(m_score)
    f1=f1_score(y_test, model.predict(X_test))
    msg = "%s: train = %.3f (%.3f) / test = %.3f / f1=%.3f" % (name, cv_results.mean(), cv_results.std(), m_score, f1)
    msg_row.append(msg)
    print(msg)
    
#Диаграмма размаха («ящик с усами»)
fig = pyplot.figure()
fig.suptitle('Сравнение результатов выполнения алгоритмов')
ax = fig.add_subplot(111)
red_square = dict(markerfacecolor='r', marker='s')
pyplot.boxplot(results, flierprops=red_square)
ax.set_xticklabels(names, rotation=45)
pyplot.show()

# Задача линейной регрессии<a class="anchor" id="8"></a>

Для решения проблемы прогнозирования задействуем следующие алгоритмы:

## Линейные алгоритмы<a class="anchor" id="8-1"></a>

- Линейная регрессия / Linear Regression (‘LR’)
- Гребневая регрессия (ридж-регрессия) / Ridge Regression (‘R’)
- Лассо-регрессия (от англ. LASSO — Least Absolute Shrinkage and Selection Operator) / Lasso Regression (‘L’)
- Метод регрессии «Эластичная сеть» / Elastic Net Regression (‘ELN’)
- Метод наименьших углов / Least Angle Regression (LARS) (‘LARS’)
- Байесовская гребневая регрессия / Bayesian ridge regression (‘BR’)

## Нелинейные алгоритмы<a class="anchor" id="8-2"></a>

- Метод k-ближайших соседей (регрессия) / k-nearest neighbors regressor (‘KNR’)
- Деревья регрессии / Decision Tree Regressor (‘DTR’)
- Линейный метод опорных векторов (регрессия) / Linear Support Vector Machine – Regression / (‘LSVR’)
- Метод опорных векторов (регрессия) / Epsilon-Support Vector Regression (‘SVR’)

## Ансамблевые алгоритмы<a class="anchor" id="8-3"></a>

- AdaBoost (регрессия) / AdaBoost Regressor (‘ABR’) (AdaBoost = Adaptive Boosting)
- Bagging (регрессия) / Bagging Regressor (‘BR’) (Bagging = Bootstrap aggregating)
- Экстра-деревья (регрессия) / Extra Trees Regressor (‘ETR’)
- Градиентный boosting (регрессия) / Gradient Boosting Regressor (‘GBR’)
- Случайный лес (регрессия) / Random Forest Classifier (‘RFR’)


In [ ]:
col_r=['Код дороги', 'Тип виновного предприятия', 'Сторонняя организация',
       'Вид работ', 'Время расстройства маневровой работы', 'Регион',
       'Погибло всего', 'Номер пути', 'Время полного перерыва движения',
       'День недели', 'Общее время задержки', 'Ранено тяжело',
       'Признак столкновения', 'Километр', 'Признак схода', 'Ранено легко',
       'Время суток', 'Месяц события',
       'Количество задержанных поездов']

Y=data['Итоговый суммарный ущерб (тыс.руб.)']
X=data[col_r]
from sklearn.model_selection import train_test_split
#разделим набор на тренировочный и тестовый
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size=0.2, random_state=42)

In [ ]:
from sklearn.linear_model import LinearRegression
from sklearn.linear_model import Ridge
from sklearn.linear_model import Lasso
from sklearn.linear_model import ElasticNet
from sklearn.linear_model import LarsCV
from sklearn.linear_model import BayesianRidge
from sklearn.neighbors import KNeighborsRegressor
from sklearn.tree import DecisionTreeRegressor
from sklearn.svm import LinearSVR
from sklearn.svm import SVR
from sklearn.ensemble import AdaBoostRegressor
from sklearn.ensemble import BaggingRegressor
from sklearn.ensemble import ExtraTreesRegressor
from sklearn.ensemble import GradientBoostingRegressor
from sklearn.ensemble import RandomForestRegressor
from sklearn.model_selection import KFold
from sklearn.model_selection import cross_val_score
from sklearn.pipeline import Pipeline
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import Normalizer
from sklearn.linear_model import Lars
from matplotlib import pyplot

num_folds = 15
n_iter = 1000
n_estimators = 100
scoring = 'r2'

models = []
models.append(('LR', LinearRegression()))
models.append(('R', Ridge()))
models.append(('L', Lasso()))
models.append(('ELN', ElasticNet()))
models.append(('LARS', Lars()))
models.append(('BR', BayesianRidge(n_iter=n_iter)))
models.append(('KNR', KNeighborsRegressor()))
models.append(('DTR', DecisionTreeRegressor()))
models.append(('LSVR', LinearSVR()))
models.append(('SVR', SVR()))
models.append(('ABR', AdaBoostRegressor(n_estimators=n_estimators)))
models.append(('BR', BaggingRegressor(n_estimators=n_estimators)))
models.append(('ETR', ExtraTreesRegressor(n_estimators=n_estimators)))
models.append(('GBR', GradientBoostingRegressor(n_estimators=n_estimators)))
models.append(('RFR', RandomForestRegressor(n_estimators=n_estimators)))

#Оценивание эффективности выполнения каждого алгоритма
scores = []
names = []
results = []
predictions = []
msg_row = []
for name, model in models:
    kfold = KFold(n_splits=num_folds, random_state=seed, shuffle=True)
    cv_results = cross_val_score(model, X_train, y_train, cv=kfold, scoring=scoring)
    names.append(name)
    results.append(cv_results)
    m_fit = model.fit(X_train, y_train)
    m_predict = model.predict(X_test)
    predictions.append(m_predict)
    m_score = model.score(X_test, y_test)
    scores.append(m_score)
    msg = "%s: train = %.3f (%.3f) / test = %.3f" % (name, cv_results.mean(), cv_results.std(), m_score)
    msg_row.append(msg)
    print(msg)
    
#Диаграмма размаха («ящик с усами»)
fig = pyplot.figure()
fig.suptitle('Сравнение результатов выполнения алгоритмов')
ax = fig.add_subplot(111)
red_square = dict(markerfacecolor='r', marker='s')
pyplot.boxplot(results, flierprops=red_square)
ax.set_xticklabels(names, rotation=45)
pyplot.show()